<a href="https://colab.research.google.com/github/canislatranscoxus/tensorflow/blob/master/3_keras_sequential_taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducing the Keras Sequential API

## flowers classification


https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/3_keras_sequential_api.ipynb


In [3]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

chown: invalid user: ‘jupyter:jupyter’


In [4]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1 || pip install tensorflow==2.1

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 43.7MB/s 
     |████████████████████████████████| 3.9MB 37.1MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=2e9ac214832e1592438e285e397ad4511347fd0d3780750d2be394aee4757187
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

## import libraries

In [5]:
import datetime
import os
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.keras.callbacks import TensorBoard

print(tf.__version__)
%matplotlib inline

2.1.0


## load data

In [6]:
train_dataset_url = "https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/data/taxi-train.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

2015232/Unknown - 0s 0us/stepLocal copy of the dataset file: /root/.keras/datasets/taxi-train.csv


### download csv files

In [47]:
# donwload csv datasset function

def download_csv( dataset_url ):

  dataset_fp = tf.keras.utils.get_file(fname=os.path.basename( dataset_url ),
                                           origin= dataset_url)
  print("Local copy of the dataset file: {}".format(train_dataset_fp))
  return dataset_fp


In [49]:
tmp_url = "https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/data/taxi-{}.csv"

# file paths
train_dataset_fp = download_csv( tmp_url.format( 'train' ) )
valid_dataset_fp = download_csv( tmp_url.format( 'valid' ) )
test_dataset_fp  = download_csv( tmp_url.format( 'test'  ) )


Local copy of the dataset file: /root/.keras/datasets/taxi-train.csv
Local copy of the dataset file: /root/.keras/datasets/taxi-train.csv
Local copy of the dataset file: /root/.keras/datasets/taxi-train.csv


In [50]:
!ls -l /root/.keras/datasets/*.csv

-rw-r--r-- 1 root root  732133 Sep  3 22:18 /root/.keras/datasets/taxi-test.csv
-rw-r--r-- 1 root root 2008310 Sep  3 21:41 /root/.keras/datasets/taxi-train.csv
-rw-r--r-- 1 root root  731341 Sep  3 22:18 /root/.keras/datasets/taxi-valid.csv


In [51]:
!head /root/.keras/datasets/taxi-train.csv






<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">


##remove unwanted columns

In [52]:
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]
UNWANTED_COLS = ['pickup_datetime', 'key']

In [55]:
def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label


In [56]:
def create_dataset(pattern, batch_size=1, mode='eval'):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels)

    if mode == 'train':
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

## crate features

In [57]:
INPUT_COLS = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
]

# Create input layer of feature columns
# TODO 1
feature_columns = {
    colname: tf.feature_column.numeric_column(colname)
    for colname in INPUT_COLS
    }

## crate model

In [58]:
# Build a keras DNN model using Sequential API
# TODO 2a
model = Sequential([
    DenseFeatures(feature_columns=feature_columns.values()),
    Dense(units = 32, activation = "relu"  , name= "h1"     ),
    Dense(units =  8, activation = "relu"  , name= "h2"     ),
    Dense(units =  1, activation = "linear", name= "output" )
    ])

In [59]:
# TODO 2b
# Create a custom evalution metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


# Compile the keras model
model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

## train model

In [60]:
TRAIN_BATCH_SIZE   = 1000
NUM_TRAIN_EXAMPLES = 10000 * 5  # training dataset will repeat, wrap around
NUM_EVALS          = 50         # how many times to evaluate
NUM_EVAL_EXAMPLES  = 10000      # enough to get a reasonable sample



In [63]:
trainds = create_dataset(
    pattern    = train_dataset_fp,
    batch_size = TRAIN_BATCH_SIZE,
    mode       = 'train')

evalds = create_dataset(
    pattern     = valid_dataset_fp,
    batch_size  = 1000,
    mode        = 'eval' ).take(NUM_EVAL_EXAMPLES//1000)

In [64]:
%time 
# TODO 3
steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)

LOGDIR = "./taxi_trained"
history = model.fit(x=trainds
                    ,steps_per_epoch    = steps_per_epoch
                    ,epochs             = NUM_EVALS
                    ,validation_data    = evalds
                    #,callbacks=[TensorBoard(LOGDIR)]
                    )

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


InvalidArgumentError: ignored

## save model to file - serialization

## load model from file

## make predictions

## see model performance